## **Import Libraries**

In [1]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn import datasets
import os
import math
import torch
import numpy as np

## **Data Processing**

In [2]:
def train_test_split(X, y, test_size, random_state=None):
  data_num = len(y)
  test_num = int(test_size * data_num)
  if random_state != None: np.random.seed(random_state)
  index = np.random.permutation(data_num)
  X_train = X[index[:-test_num]]
  X_test = X[index[-test_num:]]
  y_train = y[index[:-test_num]]
  y_test = y[index[-test_num:]]
  return X_train, X_test, y_train, y_test

In [3]:
wine_X, wine_y = datasets.load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(wine_X, wine_y, 0.2, 123)

In [4]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

## **Implementation**

In [5]:
def gaussian_likelihood(X, mu, sigma):
  """
  Computes the Gaussian pdf

  Parameters
  ----------
  X: Features of tabular data 
    Torch tensor 
  mu: Mean parameter of Gaussian likelihood
    Float
  sigma: Standard deviation of Gaussian likelihood
    Float
  
  Return
  ------
  Gaussian pdf
  """
  return (2 * np.pi * sigma ** 2) ** (-0.5) * torch.exp((- (X - mu) ** 2) / (2 * sigma ** 2))

In [6]:
def accuracy_score(y_true, y_pred):
    score = sum(y_true == y_pred)/len(y_true)
    if 
    return round(score.numpy().item(), 3)

In [7]:
is_categorical = [0] * X_train.size(1)
num_feats = X_train.size(1)
# size = X.size(0)
y_vals_unique = y_train.unique()
num_class = len(y_vals_unique)
# Probability of each class in the training set
class_probs = y_train.int().bincount().float()/len(y_train)

feats_vals_max = torch.zeros((num_feats,), dtype = torch.int32)
for i in range(num_feats):
  feats_vals_max[i] = X_train[:, i].max()
# Initialize list to store p(x_j | c_i)
likelihoods = []
for i in range(num_class):
  likelihoods.append([])
  # Index to group samples by class
  idx = torch.where(y_train == y_vals_unique[i])[0] # torch.where returns a tuple 
  curr_class = X_train[idx]
  class_size = curr_class.size(0)
  for j in range(num_feats):
    # Store all classes
    likelihoods[i].append([])
    if is_categorical[j]:
      for k in range(feats_vals_max[j] + 1):
        # Count number of observations of each feature given the class
        prob_feat_in_class = (torch.where(curr_class[:, j])[0].size(0) + 1) / class_size
        likelihoods[i][j].append(prob_feat_in_class)

    else:
      feats_class = curr_class[:, j]
      mean = feats_class.mean()
      sigma = feats_class.std() # set 'correction = 0' for not using Bessel's correction
      likelihoods[i][j] = [mean, sigma]

num_obs = X_test.size(0)
pred = torch.zeros((num_obs, num_class), dtype = torch.float32)
for k in range(num_obs):
  curr_obs = X_test[k]
  for i in range(num_class):
    pred[k][i] = class_probs[i] # Set prior probability for ith class p(c_i)
    prob_feat_in_class = likelihoods[i] # likelihoods for ith class p(x_j | c_i)
    for j in range(num_feats):
      if is_categorical[j]:
        pred[k][i] *= prob_feat_in_class[j][curr_obs[j].int()]
      else:
        mean, sigma = prob_feat_in_class[j]
        pred[k][i] *= gaussian_likelihood(curr_obs[j], mean, sigma)
  
label_pred = pred.argmax(dim = 1)

## **Results**

In [8]:
likelihoods

[[[tensor(13.7522), tensor(0.4796)],
  [tensor(1.9955), tensor(0.6775)],
  [tensor(2.4547), tensor(0.2328)],
  [tensor(16.9706), tensor(2.5946)],
  [tensor(105.8627), tensor(10.6396)],
  [tensor(2.8325), tensor(0.3560)],
  [tensor(2.9539), tensor(0.4053)],
  [tensor(0.2892), tensor(0.0681)],
  [tensor(1.8657), tensor(0.4109)],
  [tensor(5.4451), tensor(1.1998)],
  [tensor(1.0647), tensor(0.1175)],
  [tensor(3.1739), tensor(0.3651)],
  [tensor(1120.1372), tensor(229.6118)]],
 [[tensor(12.2321), tensor(0.5344)],
  [tensor(1.8669), tensor(0.8681)],
  [tensor(2.2217), tensor(0.3329)],
  [tensor(20.2481), tensor(3.4085)],
  [tensor(94.0192), tensor(15.8714)],
  [tensor(2.2037), tensor(0.5135)],
  [tensor(1.9908), tensor(0.6758)],
  [tensor(0.3648), tensor(0.1146)],
  [tensor(1.5427), tensor(0.5738)],
  [tensor(3.0338), tensor(0.9499)],
  [tensor(1.0903), tensor(0.1983)],
  [tensor(2.7702), tensor(0.4867)],
  [tensor(516.5961), tensor(151.8240)]],
 [[tensor(13.0988), tensor(0.5300)],
  [tens

In [9]:
label_pred

tensor([1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 1, 0, 0, 0, 1, 2, 1, 2, 2, 1, 0, 1, 0, 2,
        1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1])

## **Compare with Sklearn GaussianNB function**

In [19]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)
y_pred_sklearn = model.predict(X_test)
y_pred_sklearn = torch.from_numpy(y_pred_sklearn).float()
accuracy_score(y_test, y_pred_sklearn)

0.971